In [1]:
import requests
import pandas as pd
import time

# 1. Получаем список всех USDC-пар на Kraken
pair_resp = requests.get('https://api.kraken.com/0/public/AssetPairs').json()
usdc_pairs = []
for key, val in pair_resp['result'].items():
    if val.get('wsname') and '/USDC' in val['wsname'] and '.d' not in key:
        usdc_pairs.append({'pair': key, 'wsname': val['wsname'], 'altname': val['altname']})

print(f"Всего найдено {len(usdc_pairs)} пар к USDC на Kraken.")

# 2. Проверяем ликвидность и спред
pre_filtered = []
for p in usdc_pairs:
    ticker_url = f"https://api.kraken.com/0/public/Ticker?pair={p['pair']}"
    resp = requests.get(ticker_url).json()
    result = list(resp['result'].values())[0]
    bid = float(result['b'][0])
    ask = float(result['a'][0])
    spread = (ask - bid) / ((ask + bid) / 2) * 100
    volume_24h = float(result['v'][1])
    if spread < 0.2 and volume_24h > 100000:
        pre_filtered.append({**p, 'spread_%': spread, 'volume_24h': volume_24h, 'bid': bid, 'ask': ask})

# 3. Проверяем реальную активность за последний час
summary = []
for p in pre_filtered:
    try:
        trades_url = f"https://api.kraken.com/0/public/Trades?pair={p['altname']}"
        resp = requests.get(trades_url).json()
        trades = list(resp['result'].values())[0]
        since = time.time() - 3600
        trades_last_hour = [t for t in trades if float(t[2]) > since]
        vol = sum(float(t[1]) for t in trades_last_hour)
        ntrades = len(trades_last_hour)
        summary.append({**p, 'trades_last_hour': ntrades, 'real_volume_last_hour': vol})
    except Exception as e:
        print(f"Ошибка с парой {p['wsname']}: {e}")

# 4. Формируем DataFrame и выбираем топ-3 по реальному обороту за час
df = pd.DataFrame(summary)
if df.empty:
    print("Нет подходящих пар по всем критериям!")
else:
    df = df.sort_values('real_volume_last_hour', ascending=False)
    top3 = df.head(3)[['wsname', 'spread_%', 'volume_24h', 'trades_last_hour', 'real_volume_last_hour']]
    print("=== Топ-3 пары для торговли по критериям ликвидности, спреда и реального оборота ===")
    display(top3)


Всего найдено 38 пар к USDC на Kraken.
=== Топ-3 пары для торговли по критериям ликвидности, спреда и реального оборота ===


,wsname,spread_%,volume_24h,trades_last_hour,real_volume_last_hour
0,XRP/USDC,0.00041,228253.208938,126,56964.048807
